Sentiment Analyzer for Shopee Reviews

In [10]:
import numpy as np 
import pandas as pd 
import nltk
import subprocess
from nltk.corpus import wordnet


Caching the list of root modules, please wait!
(This will only be done once - type '%rehashx' to reset cache!)



In [12]:
import re
from bs4 import BeautifulSoup
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score
from nltk.sentiment import SentimentIntensityAnalyzer
sia = SentimentIntensityAnalyzer()

In [13]:
data = pd.read_csv('data/SHOPEE_REVIEWS.csv', index_col=0)

In [22]:
data = data.dropna()
nltk.download('wordnet')

[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\ryku1\AppData\Roaming\nltk_data...


True

In [18]:
def clean_text(text):
    # Remove HTML tags
    text = BeautifulSoup(text, "html.parser").get_text()

    # Convert to lowercase
    text = text.lower()

    # Remove URLs
    text = re.sub(r'http\S+', '', text)

    # Remove non-alphanumeric characters and punctuation
    text = re.sub(r'[^a-zA-Z\s]', '', text)
    
    # Remove emojis
    text = re.sub(r'[\U00010000-\U0010ffff]', '', text)

    # Tokenization
    tokens = word_tokenize(text)

    # Remove stopwords
    stop_words = set(stopwords.words('english'))
    tokens = [word for word in tokens if word.lower() not in stop_words]

    # Lemmatization
    lemmatizer = WordNetLemmatizer()
    tokens = [lemmatizer.lemmatize(word) for word in tokens]

    # Remove short words
    tokens = [word for word in tokens if len(word) > 2]

    # Join tokens back into a single string
    cleaned_text = ' '.join(tokens)

    return cleaned_text

In [23]:
data['cleaned_text'] = data['review_text'].apply(clean_text)

C:\Users\ryku1\AppData\Local\Temp\ipykernel_3840\2713013487.py:3: MarkupResemblesLocatorWarning: The input looks more like a filename than markup. You may want to open this file and pass the filehandle into Beautiful Soup.
  text = BeautifulSoup(text, "html.parser").get_text()


In [24]:
# Sentiment Analyzer
data['Sentiment Scores'] = data['cleaned_text'].apply(lambda text: sia.polarity_scores(text))
data['Compound Sentiment Score'] = data['Sentiment Scores'].apply(lambda scores: scores['compound'])
data['pos'] = data['Sentiment Scores'].apply(lambda scores: scores['pos'])
data['neg'] = data['Sentiment Scores'].apply(lambda scores: scores['neg'])
data['neu'] = data['Sentiment Scores'].apply(lambda scores: scores['neu'])

In [25]:
X = data[['Compound Sentiment Score','pos','neg','neu']]
Y = data['review_rating']

In [42]:
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.2, random_state=32)

In [43]:
RForest_clf = RandomForestClassifier(n_estimators = 100)
RForest_clf.fit(X_train, y_train)
y_pred = RForest_clf.predict(X_test)
accuracy = accuracy_score(y_test, y_pred)*100
print(f'Accuracy: {accuracy}')

Accuracy: 79.3225111671681
